# Preprocessing

In [1]:
import pandas as pd
import numpy as np
import os
import yaml
from tqdm.notebook import trange, tqdm

node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

dataset_name = "211110_anewbeginning"
mapping_path = f"{base_path}/data/mapping"
dataset_path = f"{base_path}/data/2_datasets_pre/{dataset_name}"

/sc-projects/sc-proj-ukb-cvd


In [2]:
from pathlib import Path
Path(dataset_path).mkdir(parents=True, exist_ok=True)

In [3]:
vocab_dir = f"{mapping_path}/athena"
concept = pd.read_csv(f"{vocab_dir}/CONCEPT.csv", sep='\t')

/home/buergelt/miniconda3/envs/ehrgraphs2/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (5,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
sct_icd = pd.read_csv(f"{mapping_path}/umls/der2_iisssccRefset_ExtendedMapFull_US1000124_20220301.txt", sep="\t")

In [5]:
sct_icd

,id,effectiveTime,active,moduleId,refsetId,referencedComponentId,mapGroup,mapPriority,mapRule,mapAdvice,mapTarget,correlationId,mapCategoryId
0,80004cdf-a114-59c8-88a2-b6f0283acf4f,20180301,1,5991000124107,6011000124106,128041000119107,1,1,IFA 721617001 | Primary adenocarcinoma of lowe...,IF PRIMARY ADENOCARCINOMA OF LOWER THIRD OF ES...,K22.70,447561005,447639009
1,80005aeb-477c-53dc-9a5c-ce723ca264cb,20150731,1,449080006,447562003,254153009,1,1,TRUE,ALWAYS Q79.8,Q79.8,447561005,447637006
2,80007b64-5a60-5556-88ce-22ef540a3ea5,20180301,1,5991000124107,6011000124106,281801000009108,2,2,OTHERWISE TRUE,ALWAYS B96.89,B96.89,447561005,447637006
3,80007e6a-7408-5b87-a1ec-70b212811410,20190731,1,449080006,447562003,16623961000119100,1,1,TRUE,ALWAYS D61.1,D61.1,447561005,447637006
4,80009454-5531-5f78-b7c9-d288f2346d83,20150731,1,449080006,447562003,301327002,1,1,TRUE,MAP SOURCE CONCEPT CANNOT BE CLASSIFIED WITH A...,NaN,447561005,447638001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
696187,7fff513a-5f56-5d95-94a4-ad3ca8799d7e,20170901,1,5991000124107,6011000124106,405779006,1,1,TRUE,ALWAYS Z76.89,Z76.89,447561005,447637006
696188,7fff619f-e8b2-55c3-b72b-0e8bc94e9b51,20180301,0,5991000124107,6011000124106,699317002,1,3,IFA 432131000124100 | Malignant mesothelioma o...,IF MALIGNANT MESOTHELIOMA OF THE TUNICA VAGINA...,C45.7,447561005,447639009
696189,7fff797f-2343-5e27-ad05-83c727866186,20130731,1,449080006,447562003,94152006,1,1,TRUE,ALWAYS C78.8 | POSSIBLE REQUIREMENT FOR MORPHO...,C78.8,447561005,447637006
696190,7fffb435-c140-5f1a-b314-7d228b634825,20130731,1,449080006,447562003,80736008,1,1,TRUE,ALWAYS K60.4 | DESCENDANTS NOT EXHAUSTIVELY MA...,K60.4,447561005,447637006


In [6]:
relationship = pd.read_csv(f"{vocab_dir}/RELATIONSHIP.csv", sep='\t')

In [7]:
vocabulary = pd.read_csv(f"{vocab_dir}/VOCABULARY.csv", sep='\t')

In [8]:
# links omop concept_ids
concept_relationship = pd.read_csv(f"{vocab_dir}/CONCEPT_RELATIONSHIP.csv", sep='\t')

## Prepare Records TTE

In [4]:
import wandb
import pathlib

# project="recordgraphs"
project="retina"
entity="cardiors"
artifact_date = "220812"

In [5]:
def get_path_from_wandb(reference: str):
    path = pathlib.Path(reference.split("file://")[1])
    assert path.exists()
    return path

def read_artifact(run, artifact_name, type_name):
    
    artifact = run.use_artifact(f'cardiors/recordgraphs/{artifact_name}:latest', type=type_name)
    file_path = get_path_from_wandb(artifact.manifest.entries[artifact_name].ref)
    print(file_path)

    return file_path

In [6]:
def add_artifact(run, artifact_name, type_name, description, artifact_path, df):
   
    artifact = wandb.Artifact(artifact_name, type=type_name, 
                              description=description,
                             metadata = {"1_shape": f"{len(df)}x{len(df.columns)}",
                                         "2_cols": str(df.columns.to_list())})
    artifact.add_reference(f"""file://{artifact_path}""", artifact_name, checksum=True)
    run.log_artifact(artifact)

In [7]:
run = wandb.init(project=project, job_type="log_artifact", entity=entity, tags=["artifact"])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cardiors (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [ ]:
phecode_defs_path = "/sc-projects/sc-proj-ukb-cvd/data/mapping/phecodes/phecode_strings_V2.csv"

artifact_name = "phecode_definitions"
type_name = "prepare_phecodes"
description = "Definition of phecodes"
artifact_path = f"{dataset_path}/artifacts/{artifact_name}_{artifact_date}.feather"

phecode_defs = pd.read_csv(phecode_defs_path, dtype={"phecode": str}).sort_values("phecode").reset_index()
phecode_defs.to_feather(artifact_path)

add_artifact(run, artifact_name, type_name, description, artifact_path, phecode_defs)

In [ ]:
phecode_mapping_path = "/sc-projects/sc-proj-ukb-cvd/data/mapping/phecodes/ICD10_to_phecode_V2.csv"

artifact_name = "icd10_phecode_mapping"
type_name = "prepare_phecodes"
description = "Mapping of ICD10 to phecodes"
artifact_path = f"{dataset_path}/artifacts/{artifact_name}_{artifact_date}.feather"

phecode_mapping = pd.read_csv(phecode_mapping_path, dtype={"phecode": str}).rename(columns={"icd10": "ICD10"})\
    .assign(ICD10 = lambda x: x.ICD10.str.replace(".", "", regex=False)).reset_index()
phecode_mapping.to_feather(artifact_path)

add_artifact(run, artifact_name, type_name, description, artifact_path, phecode_mapping)

In [ ]:
records_raw = pd.read_feather(read_artifact(run, "complete_records_extended", "prepare_records"))\
    .query("concept_id==concept_id")\
    .assign(concept_id = lambda x: x.concept_id.astype(int))
records_raw.origin.unique()

In [ ]:
1+1

### Explicitly encode death record

In [ ]:
records_death = records_raw.query('origin=="death_records"').copy()
#records_death.loc[records_death.query('origin=="death_records"').index, 'code'] = 4306655
records_death.loc[records_death.query('origin=="death_records"').index, 'concept_id'] = 4306655
records_death = records_death.drop_duplicates(subset=['eid', 'concept_id'], keep='first')
records_raw_w_death = pd.concat([records_raw, records_death], axis=0)

In [ ]:
records_raw_w_death

### Prepare OMOP

In [ ]:
records_omop = records_raw_w_death.assign(concept_code=lambda x: x.concept_id, vocabulary="OMOP")
records_omop["vocabulary"] = records_omop["vocabulary"].astype("category")

In [ ]:
records_omop.head()

### Prepare OMOP -> ICD10

In [ ]:
omop_counts = records_raw[["concept_id"]].value_counts()

In [ ]:
omop_concepts = omop_counts.to_frame().reset_index()[["concept_id"]]
omop_concepts_md = omop_concepts.merge(concept[["concept_id", "concept_code", "concept_name", "vocabulary_id", "domain_id", "concept_class_id"]], on="concept_id", how="left").merge(omop_counts.to_frame().reset_index(), on="concept_id", how="left")

In [ ]:
omop_concepts_md

In [ ]:
omop_counts

In [ ]:
sct_icd

In [ ]:
def map_omop_icd10():
    concepts_in = omop_concepts.concept_id.to_list()
    concepts_out = concept.query("vocabulary_id=='ICD10'").concept_id.to_list()
    mapping = concept_relationship.query("concept_id_1==@concepts_in&concept_id_2==@concepts_out")\
        .merge(concept, left_on="concept_id_2", right_on="concept_id")
    mapping_clean = mapping[["concept_id_1", "concept_code"]]\
        .rename(columns={"concept_id_1":"concept_id"}).assign(vocabulary="ICD10")\
        .assign(concept_code = lambda x: x.concept_code.str.replace(".", "", regex=False))
    return mapping_clean

In [ ]:
mapping_omop_icd10 = map_omop_icd10()

In [ ]:
mapping_omop_icd10#.query("relationship_id=='Value mapped from'")

### Prepare ICD10 -> Phecodes

In [ ]:
icd10_phecode_mapping = phecode_mapping.rename(columns={"phecode": "concept_code"})[["ICD10", "concept_code"]].assign(vocabulary="phecode")

In [ ]:
mapping_icd10_phecode = mapping_omop_icd10.rename(columns={"concept_code": "ICD10"})[["concept_id", "ICD10"]]\
    .merge(icd10_phecode_mapping, on="ICD10")[["concept_id", "concept_code", "vocabulary"]]

In [ ]:
mappings_omop_icd10_phecode = pd.concat([mapping_omop_icd10, mapping_icd10_phecode], axis=0).reset_index(drop=True)
mappings_omop_icd10_phecode["concept_code"] = mappings_omop_icd10_phecode["concept_code"].astype("category")
mappings_omop_icd10_phecode["vocabulary"] = mappings_omop_icd10_phecode["vocabulary"].astype("category")
mappings_omop_icd10_phecode = mappings_omop_icd10_phecode.set_index("concept_id")

In [ ]:
mappings_omop_icd10_phecode

In [ ]:
mappings_omop_icd10_phecode.dtypes

### Join ICD10 and phecode mappings

In [ ]:
records_icd10_phecode = records_raw_w_death.set_index("concept_id")\
    .merge(mappings_omop_icd10_phecode, left_index=True, right_index=True)

In [ ]:
records_icd10_phecode = records_icd10_phecode.reset_index()\
    .query("concept_code==concept_code")\
    .drop_duplicates(subset=["eid", "date", "concept_code", "vocabulary", "origin"], keep="first")

### Merge all mapped records

In [ ]:
records_all_vocabs = pd.concat([records_omop, records_icd10_phecode], axis=0).reset_index(drop=True)

In [ ]:
records_all_vocabs.info()

In [ ]:
records_mapped_unique = records_all_vocabs.assign(concept_id=lambda x: x.vocabulary +"_" +x.concept_code.astype(str)).reset_index(drop=True).drop(columns=["concept_code"])

In [ ]:
# make everything faster
for col in tqdm(records_mapped_unique.columns): 
    if "date" in col: records_mapped_unique[col] = records_mapped_unique[col].astype("datetime64[D]")

In [ ]:
# make everything faster
for col in tqdm(["domain_id", "code", "concept_id", "origin", "vocabulary"]): 
    records_mapped_unique[col] = records_mapped_unique[col].astype("category")

In [ ]:
records_mapped_unique = records_mapped_unique.drop_duplicates().reset_index(drop=True)

In [ ]:
records_mapped_unique.shape

In [ ]:
records_mapped_unique.info()

In [ ]:
def add_artifact(run, artifact_name, type_name, description, artifact_path, df):
   
    artifact = wandb.Artifact(artifact_name, type=type_name, 
                              description=description,
                             metadata = {"1_shape": f"{len(df)}x{len(df.columns)}",
                                         "2_cols": str(df.columns.to_list())})
    artifact.add_reference(f"""file://{artifact_path}""", artifact_name, checksum=True)
    run.log_artifact(artifact)

In [ ]:
# log in wandb
artifact_name = "final_records_omop"
type_name = "prepare_records"
description = "Cleaned first patient records from the data portal in long format"
artifact_path = f"{dataset_path}/artifacts/{artifact_name}_{artifact_date}.feather"
records_mapped_unique.to_feather(artifact_path)
add_artifact(run, artifact_name, type_name, description, artifact_path, records_mapped_unique)

# Upload baseline file

In [10]:
baseline_outcomes = '/sc-projects/sc-proj-ukb-cvd/results/projects/22_retina_phewas/data/baseline_outcomes_220627.feather'
bo_df = pd.read_feather(baseline_outcomes)


In [12]:
bo_df.head()

,eid,OMOP_4306655_prev,phecode_202_prev,phecode_401_prev,phecode_401-1_prev,phecode_475_prev,phecode_202-2_prev,phecode_718_prev,phecode_089_prev,phecode_130_prev,...,phecode_247-85_time,phecode_361-24_time,phecode_424-7_time,phecode_244-3_time,phecode_824-1_time,phecode_206-1_time,phecode_018_time,phecode_363-8_time,phecode_396-6_time,phecode_361-93_time
0,1000018,False,False,True,True,False,False,False,False,False,...,11.866089,11.866089,11.866089,11.866089,11.866089,11.866089,11.866089,11.866089,11.866089,11.866089
1,1000020,False,False,False,False,False,False,False,False,False,...,13.596446,13.596446,13.596446,13.596446,13.596446,13.596446,13.596446,13.596446,13.596446,13.596446
2,1000037,False,False,False,False,False,False,True,False,False,...,12.868163,12.868163,12.868163,12.868163,12.868163,12.868163,12.868163,12.868163,12.868163,12.868163
3,1000043,False,False,True,True,False,False,False,True,True,...,12.309629,12.309629,12.309629,12.309629,12.309629,12.309629,12.309629,12.309629,12.309629,12.309629
4,1000051,False,True,False,False,False,True,False,False,False,...,15.291210,15.291210,15.291210,15.291210,15.291210,15.291210,15.291210,15.291210,15.291210,15.291210


In [13]:
artifact_name = "baseline_outcomes"
type_name = "baseline_file"
description = "Clean outcomes from dataportal at time of recruitment"
artifact_path = f"{dataset_path}/artifacts/{artifact_name}_220627.feather"
bo_df.to_feather(artifact_path)
add_artifact(run, artifact_name, type_name, description, artifact_path, bo_df)

wandb: WARNING Serializing object of type str that is 189087 bytes


In [ ]:
artifact = run.use_artifact('cardiors/retina/eids:v0', type='prepare_splits')
artifact_dir = artifact.download()
artifact_dir

In [16]:
def add_artifact(run, artifact_name, type_name, description, artifact_path, df):
   
    artifact = wandb.Artifact(artifact_name, type=type_name, 
                              description=description, metadata = {"1_shape": "manual upload"})
    artifact.add_reference(f"""file://{artifact_path}""", artifact_name, checksum=True)
    run.log_artifact(artifact)

In [13]:
import yaml
eids_path = '/sc-projects/sc-proj-ukb-cvd/data/2_datasets_pre/211110_anewbeginning/artifacts/eids_211209.yaml'
eid_dict = yaml.load(open(eids_path), Loader=yaml.CLoader)

In [14]:
run = wandb.init(project=project, job_type="log_artifact", entity=entity, tags=["artifact"])

wandb: wandb version 0.12.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [17]:
artifact_name = "eids"
type_name = "prepare_splits"
description = "Dictionary containing the eids based on the assessment centres and train, valid and test splits"
# artifact_path = f"{dataset_path}/artifacts/{artifact_name}_211209.feather"
add_artifact(run, artifact_name, type_name, description, eids_path, eid_dict)

In [ ]:
eid_dict = '/sc-projects/sc-proj-ukb-cvd/data/2_datasets_pre/211110_anewbeginning/artifacts/eids:v0'


In [18]:
# add covariate file

In [13]:
artifact = run.use_artifact('cardiors/RecordGraphs/baseline_covariates:latest', type='prepare_covariates')
artifact_dir = artifact.download()
artifact_dir

'./artifacts/baseline_covariates:v2'

In [19]:
x = pd.read_feather('/sc-projects/sc-proj-ukb-cvd/data/2_datasets_pre/211110_anewbeginning/baseline_covariates_220503.feather')
#                     /sc-projects/sc-proj-ukb-cvd/data/2_datasets_pre/211110_anewbeginning/baseline_covariates_220323.feather')

wandb: Network error (ReadTimeout), entering retry loop.


In [16]:
def add_artifact(run, artifact_name, type_name, description, artifact_path, df):
   
    artifact = wandb.Artifact(artifact_name, type=type_name, 
                              description=description,
                             metadata = {"1_shape": f"{len(df)}x{len(df.columns)}",
                                         "2_cols": str(df.columns.to_list())})
    artifact.add_reference(f"""file://{artifact_path}""", artifact_name, checksum=True)
    run.log_artifact(artifact)

In [18]:
# log in wandb
artifact_name = "baseline_covariates"
type_name = "prepare_covariates"
description = "Dataframe of covariates at recruitment"
artifact_path = f"{dataset_path}/artifacts/{artifact_name}_220503.feather"
x.to_feather(artifact_path)
add_artifact(run, artifact_name, type_name, description, artifact_path, x)